In [1]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [2]:
root = "../raw/it.openfoodfacts.org"
foods = [food for food in os.listdir(root) if food[0] != "."]

### Merging csv files

In [84]:
def parse_food(food):
    textual = set([
        "Categorie", "Confezionamento", "Etichette, certificazioni, premi",
        "Nutrition score  France", "barcode",
        "Link alla pagina del prodotto sul sito ufficiale del produttore",
        "Quantità", "Luoghi di produzione o lavorazione", "Marche", "Negozi",
        "Nome generico", "NutriScore", "Origine degli ingredienti",
        "Paesi di vendita", "additives", "codice EMB", "ingredients", "name"
    ])
    root = "../raw/it.openfoodfacts.org"
    file_path = "{path}/{name}/Informazioni nutrizionali.csv".format(
        path=root, name=food)
    try:
        df = pd.read_csv(
            file_path,
            usecols=[
                "Informazioni nutrizionali", "Come vendutoper 100 g / 100 ml"
            ],
            index_col="Informazioni nutrizionali")
    except Exception as e:
        return None
    if (df == "?").any().any():
        return None
    for index, value in df.iterrows():
        value = value["Come vendutoper 100 g / 100 ml"]
        if isinstance(value, str):
            unit = ""
            value = value.replace(".", "").replace(",", ".").replace("<", "")
            if "mg" in value:
                value = float(value.split("mg")[0]) * 1e-3
                unit = "g"
            elif "µg" in value:
                value = float(value.split("µg")[0]) * 1e-6
                unit = "g"
            elif "g" in value:
                value = float(value.split("g")[0])
                unit = "g"
            elif "% vol" in value:
                value = float(value.split("% vol")[0])
                unit = "% vol"
            elif "%" in value:
                value = float(value.split("%")[0])
                unit = "%"
            elif re.match("-?[\dA-Z]", value):
                pass
            else:
                raise ValueError(str((value, index)))
            df.loc[index] = value
            if unit:
                df = df.rename(
                    index={
                        index:
                        "{index} | {unit}".format(index=index, unit=unit)
                        .lower().strip().replace("  ", " ")
                    })

    df = df.transpose()
    with open("{path}/{name}/metadata.json".format(path=root, name=food),
              "r") as f:
        for key, value in json.load(f).items():
            if value:
                df[key] = value.replace("\n", "; ")
    df.index = df["name"]
    df.index.name = "name"
    df.columns = [c.replace("-", "").strip() for c in df.columns]
    del df.columns.name
    df = df.astype({
        c:float
        for c in set(df.columns) - textual
    })
    return df

In [85]:
from IPython.display import display
with Pool(cpu_count()) as p:
    open_food_facts = pd.concat(
        list(tqdm(p.imap(parse_food, foods), total=len(foods))))

In [86]:
tofloat = [
    'zuccheri | g'
]
open_food_facts = open_food_facts.astype({k: float for k in tofloat})

In [87]:
open_food_facts.columns.to_series().groupby(open_food_facts.dtypes).groups

{dtype('float64'): Index(['Fruits, vegetables and nuts (estimate from ingredients list)',
        'acidi grassi monoinsaturi | g', 'acidi grassi polinsaturi | g',
        'acidi grassi saturi | g', 'acidi grassi trans | g', 'alcol | % vol',
        'amido | g', 'betaglucani | g', 'bicarbonato | g', 'calcio | g',
        'carboidrati | g', 'cloruro | g', 'cocoa (minimum) | %',
        'colesterolo | g', 'ferro | g', 'fibra alimentare | g', 'fluoro | g',
        'fosforo | g',
        'fruits, vegetables and nuts (estimate from ingredients list) | %',
        'fruits, vegetables and nuts (estimate from ingredients list) | g',
        'fruits, vegetables and nuts (minimum) | %', 'grassi | g',
        'hydrogénocarbonates | g', 'iodio | g', 'iodo | g', 'lattosio | g',
        'lcarnitin | g', 'magnesio | g', 'nitrate | g', 'nitrates | g',
        'nitrati | g', 'nitriti | g', 'pH',
        'polialcoli/polioli (alcoli degli zuccheri) | g', 'potassio | g',
        'proteine | g', 'résidu sec

In [88]:
open_food_facts.to_csv("../csv/openfoodsfacts.csv")